<a href="https://colab.research.google.com/github/Mathtzt/data-science/blob/main/apr_baseado_instancias/base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>